<a href="https://colab.research.google.com/github/nbazedong/Public_Code/blob/main/sample_text_to_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/openai/shap-e
!mv shap-e/* .
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 316 (delta 20), reused 10 (delta 10), pack-reused 286
Receiving objects: 100% (316/316), 11.70 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [6]:
!pip install setuptools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:15<00:00, 59.6MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
batch_size = 4
guidance_scale = 15.0
prompt = "a chair with dragon"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)